# Nama : Muchlas Andrey Pahlevi
# NIM  : 2209106082

# Import Library

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, callbacks
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

## Load Dataset

In [2]:
df = pd.read_csv('final_data.csv')

## Mengecek missing value dan data duplikat

In [3]:
df.isnull().sum()
duplicate = df.duplicated().sum()
print('Jumlah nilai duplikat pada data: ', duplicate)

Jumlah nilai duplikat pada data:  0


## Drop beberapa kolom yang tidak dibutuhkan

In [4]:
df = df.drop(['player', 'team', 'name', 'position'], axis=1)

# Splitting Data

In [5]:
X = df.drop(['current_value'], axis=1)
y = df['current_value']

# Membagi dataset menjadi training set dan test set (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalisasi fitur dan target

In [6]:
mm = MinMaxScaler()
X_train = mm.fit_transform(X_train)
X_test = mm.transform(X_test)
y_train = mm.fit_transform(np.array(y_train).reshape(-1, 1))
y_test = mm.transform(np.array(y_test).reshape(-1, 1))

In [7]:
X_train.shape

(7527, 17)

## Definisi Model

In [8]:
model = Sequential([
    layers.Dense(128, activation='relu', input_shape=[X_train.shape[1]], kernel_regularizer=l2(0.001)),
    layers.BatchNormalization(),

    layers.Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    layers.BatchNormalization(),

    layers.Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
    layers.BatchNormalization(),

    layers.Dense(16, activation='relu', kernel_regularizer=l2(0.001)),
    layers.BatchNormalization(),

    layers.Dense(1)
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## Kompilasi Model

In [9]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)

## Definisi callback EarlyStopping

In [10]:
es = callbacks.EarlyStopping(
    monitor = 'val_loss',
    patience = 5,
    verbose = 1,
    restore_best_weights = True
)

In [11]:
history = model.fit(
    X_train, y_train,
    epochs = 50,
    validation_split=0.2,
    callbacks=[es]
)

Epoch 1/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - loss: 0.3621 - mean_absolute_error: 0.2834 - val_loss: 0.2883 - val_mean_absolute_error: 0.3364
Epoch 2/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1628 - mean_absolute_error: 0.0820 - val_loss: 0.1891 - val_mean_absolute_error: 0.1924
Epoch 3/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1466 - mean_absolute_error: 0.0704 - val_loss: 0.1358 - val_mean_absolute_error: 0.0671
Epoch 4/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.1304 - mean_absolute_error: 0.0513 - val_loss: 0.1185 - val_mean_absolute_error: 0.0330
Epoch 5/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.1156 - mean_absolute_error: 0.0346 - val_loss: 0.1064 - val_mean_absolute_error: 0.0320
Epoch 6/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.1037 - mean_absolute_error: 0.0352 - val_loss: 0.0942 - val_mean_absolute_error: 0.0290
Epoch 7/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0915 - mean_absolute_error: 0.0288 - 

### Membagi data training menjadi training penuh dan validation set

In [12]:
X_train_full, X_val, y_train_full, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

### Melatih ulang model tanpa validation_split

In [13]:
history = model.fit(
    X_train_full, y_train_full,
    epochs=50,
    validation_data=(X_val, y_val),
    callbacks=[es]
)

Epoch 1/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0019 - mean_absolute_error: 0.0159 - val_loss: 0.0030 - val_mean_absolute_error: 0.0322
Epoch 2/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0020 - mean_absolute_error: 0.0179 - val_loss: 0.0027 - val_mean_absolute_error: 0.0261
Epoch 3/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0016 - mean_absolute_error: 0.0158 - val_loss: 0.0021 - val_mean_absolute_error: 0.0253
Epoch 4/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0015 - mean_absolute_error: 0.0165 - val_loss: 0.0019 - val_mean_absolute_error: 0.0251
Epoch 5/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0015 - mean_absolute_error: 0.0176 - val_loss: 0.0039 - val_mean_absolute_error: 0.0418
Epoch 6/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0015 - mean_absolute_error: 0.0165 - val_loss: 0.0013 - val_mean_absolute_error: 0.0153
Epoch 7/50
189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0015 - mean_absolute_error: 0.0188 - v

## Evaluasi Model

In [14]:
train_loss, train_mae = model.evaluate(X_train_full, y_train_full)
val_loss, val_mae = model.evaluate(X_val, y_val)

print(f"Training MAE: {train_mae*100:.2f}%")
print(f"Validation MAE: {val_mae*100:.2f}%")

189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0013 - mean_absolute_error: 0.0157
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0014 - mean_absolute_error: 0.0155
Training MAE: 1.53%
Validation MAE: 1.53%
